In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from env import username, password, hostname

In [2]:
# url = f'mysql+pymysql://{username}:{password}@{hostname}/curriculum_logs'
# query = """select * from logs join cohorts on logs.cohort_id = cohorts.id;"""
# df = pd.read_sql(query, url)
# df.drop("id", axis=1, inplace=True)
# df.to_csv("curriculum_logs.csv", index=False)
# df.head()

In [3]:
df = pd.read_csv("curriculum_logs.csv")
df.head(2)

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1


In [4]:
df["program_name"] = np.where(df["program_id"] == 3, "data science", "web dev")
df.head(2)

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_name
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev


In [5]:
df["date_time"] = pd.to_datetime(df['date'] + " " + df['time'])
df.head(2)

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_name,date_time
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev,2018-01-26 09:55:03
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev,2018-01-26 09:56:02


In [6]:
df = df.set_index(df["date_time"]).drop(["date", "time", "date_time"], axis=1)
df.head(2)

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_name
date_time,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
- These are the top 10 cohorts who accessed the curriculum the most
### DONE

In [39]:
df[["name", "path"]][df["name"] != "Staff"].groupby(by=["name"], sort=False).count().nlargest(n=10, columns="path", keep="first")

,path
name,
Ceres,40730
Zion,38096
Jupiter,37109
Fortuna,36902
Voyageurs,35636
Ganymede,33844
Apex,33568
Deimos,32888
Darden,32015


3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
- These students were active students and not staff. These are the 10 students whom accessed the curriculum the fewest
### DONE

In [ ]:
was_student = (df.index < df["end_date"]) & (df.index > df["start_date"])
not_staff = (df["name"] != "Staff")

df[["user_id", "path"]][was_student & not_staff].groupby(by=["user_id"], sort=False).count().nsmallest(n=10, columns="path", keep="first")

4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?


5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?


6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
- These are the topics by program that got referenced after a student graduated.
### DONE

In [57]:
df["path"][(df.index > df["end_date"]) & (df["program_name"] == "web dev") & (df["name"] != "Staff")]

date_time
2018-01-26 09:55:03                                      /
2018-01-26 09:56:02                                java-ii
2018-01-26 09:56:05    java-ii/object-oriented-programming
2018-01-26 09:56:06     slides/object_oriented_programming
2018-01-26 10:14:47                                      /
                                      ...                 
2021-04-21 14:43:09                      jquery/mapbox-api
2021-04-21 14:43:10                jquery/ajax/weather-map
2021-04-21 14:50:36                                 java-i
2021-04-21 14:50:38            java-i/introduction-to-java
2021-04-21 16:30:30                               appendix
Name: path, Length: 104557, dtype: object

In [58]:
df["path"][(df.index > df["end_date"]) & (df["program_name"] == "data science") & (df["name"] != "Staff")]

date_time
2020-01-30 08:01:31             1-fundamentals/1.1-intro-to-data-science
2020-01-30 08:01:32             1-fundamentals/modern-data-scientist.jpg
2020-01-30 08:01:32                 1-fundamentals/AI-ML-DL-timeline.jpg
2020-01-31 11:05:04                                                    /
2020-01-31 11:05:13             1-fundamentals/1.1-intro-to-data-science
                                             ...                        
2021-04-21 15:20:12                              classification/overview
2021-04-21 15:20:12    classification/classical_programming_vs_machin...
2021-04-21 15:20:12             classification/scale_features_or_not.svg
2021-04-21 15:20:14                               classification/project
2021-04-21 15:20:18                               classification/acquire
Name: path, Length: 11544, dtype: object

7. Which lessons are least accessed?
- These are the top 50 least accessed topics, but this can be adjusted based on needs
### DONE

In [50]:
df1["path"].value_counts(ascending=True).nsmallest(n=50, keep="first").to_dict()

{'appendix/professional-development/post-interview-review-form': 1,
 'cli-03-file-paths': 1,
 'cli-07-more-topics': 1,
 'spring/services': 1,
 '8-timeseries/1-overview': 1,
 '8-timeseries/2-intro-to-timeseries': 1,
 '8-timeseries/3-acquire': 1,
 '8-timeseries/4-prep': 1,
 '8-timeseries/6.1-parametric-modeling': 1,
 '8-timeseries/6.2-prophet': 1,
 '10-nlp/2-regular-expressions': 1,
 '10-nlp/exercises': 1,
 '10-nlp/3-acquire': 1,
 '11-distributed-ml/3-getting-started': 1,
 '11-distributed-ml/4-acquire': 1,
 '11-distributed-ml/6.1-prepare-part-1': 1,
 '11-distributed-ml/6.2-prepare-part-2': 1,
 '11-distributed-ml/6.3-prepare-part-3': 1,
 'content/html-css/search_index.json': 1,
 '11-distributed-ml/6.4-prepare-part-4': 1,
 '11-distributed-ml/8.1-classification': 1,
 '11-distributed-ml/exercises': 1,
 'uploads/598dc43df39e2.jpg': 1,
 'register.': 1,
 'appendix/extra-exercises': 1,
 'appendix/extra-exercises/javascript': 1,
 '2-stats/4.8-feature-scaling': 1,
 'appendix/further-reading/mysql'